In [1]:
import pandas as pd

In [2]:
script_df = pd.read_csv('../what-kind-of-bear-is-best/the-office-lines - scripts.csv')

In [3]:
script_df.head()

,id,season,episode,scene,line_text,speaker,deleted
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False


In [4]:
script_df.describe()

,id,season,episode,scene
count,59909.000000,59909.000000,59909.000000,59909.000000
mean,29955.000000,5.348178,11.558597,27.143852
std,17294.382975,2.389427,6.986208,17.860616
min,1.000000,1.000000,1.000000,1.000000
25%,14978.000000,3.000000,5.000000,14.000000
50%,29955.000000,5.000000,11.000000,25.000000
75%,44932.000000,7.000000,18.000000,37.000000
max,59909.000000,9.000000,26.000000,116.000000


In [5]:
script_df.set_index('id', inplace=True)

In [6]:
script_df.head()

,season,episode,scene,line_text,speaker,deleted
id,,,,,,
1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False
2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False
3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False
4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False
5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False


Let's see how many characters does the script have

In [7]:
characters = script_df.speaker.drop_duplicates()

In [8]:
len(characters)

797

According to this data source, the office has 797 characters, including minor guests appearances

I've always wanted to see who's got the more lines.

In [9]:
speakers_df = script_df.groupby('speaker')['speaker'].count().sort_values(ascending=False)

In [10]:
speakers_df.head()

speaker
Michael    12137
Dwight      7529
Jim         6814
Pam         5375
Andy        3968
Name: speaker, dtype: int64

seems like Michael has the most lines by a good number, followed by Dwight, Jim and Pam. That sounds about right.

### Now I want to find a set of questions, where Dwight provides an answer to them.

In [26]:
questions_df = script_df.loc[(script_df['line_text'].str.contains('.*\?$')) & (script_df.loc[script_df['line_text'].str.contains('.*\?$').index]['speaker'] != 'Dwight')]

In [27]:
questions_df.head()

,season,episode,scene,line_text,speaker,deleted
id,,,,,,
1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False
3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False
7,1,1,3,"I've, uh, I've been at Dunder Mifflin for 12 y...",Michael,False
10,1,1,3,What?,Pam,False
11,1,1,3,Any messages?,Michael,False


In [28]:
questions_df.index[:-1]

Int64Index([    1,     3,     7,    10,    11,    25,    32,    34,    37,
               39,
            ...
            59834, 59836, 59838, 59864, 59865, 59869, 59894, 59897, 59903,
            59904],
           dtype='int64', name='id', length=10097)

In [29]:
answers_df = script_df.loc[questions_df.index[:-1]+1] 

In [30]:
answers_df.head()

,season,episode,scene,line_text,speaker,deleted
id,,,,,,
2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False
4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False
8,1,1,3,Well. I don't know.,Pam,False
11,1,1,3,Any messages?,Michael,False
12,1,1,3,"Uh, yeah. Just a fax.",Pam,False


In [31]:
dwight_answers_df = answers_df.loc[answers_df['speaker'] == 'Dwight']

In [32]:
dwight_answers_df.head()

,season,episode,scene,line_text,speaker,deleted
id,,,,,,
86,1,1,19,What?,Dwight,False
88,1,1,19,Just clearing my desk. I can't concentrate.,Dwight,False
93,1,1,19,Safety violation. I could fall and pierce an o...,Dwight,False
125,1,1,26,On his mother's grave.,Dwight,False
149,1,1,30,He put my stuff in Jell-O again.,Dwight,False


In [33]:
len(dwight_answers_df)

1339

In [34]:
questions_df = questions_df.loc[dwight_answers_df.index-1]

In [35]:
questions_df.head()

,season,episode,scene,line_text,speaker,deleted
id,,,,,,
85,1,1,19,"Sure. Mr. Davis, let me call you right back. Y...",Jim,False
87,1,1,19,What are you doing?,Jim,False
92,1,1,19,Why not?,Jim,False
124,1,1,26,Can you promise that?,Stanley,False
148,1,1,30,"OK. Hold on, hold on. The judge is in session....",Michael,False


In [36]:
questions_and_answers_df = pd.concat([questions_df,dwight_answers_df])

In [37]:
questions_and_answers_df = questions_and_answers_df.sort_index()

In [38]:
questions_and_answers_df.to_csv('questions_and_answers_df.csv')